In [1]:
import json
import torch
from pprint import pformat
import numpy as np

In [2]:
anns = json.load(open('../data/vq_v2_val_anno.json'))
preds_egoloc = json.load(open('/data/gunsbrother/repos/EgoLoc/2d_retrieval/data/vq_val_vq3d.json'))
preds_inter_lit = torch.load('/data/gunsbrother/repos/vq2d-lightning/outputs/debug/2024-09-25/126347/intermediate_predictions.pt', weights_only=True)

In [3]:
pred_tree_lit = {}  # video_uid -> clip_uid -> annotation_uid -> qset_id -> prediction
for ann in anns:
    video_uid = ann['video_uid']
    clip_uid = ann['clip_uid']
    annotation_uid = ann['annotation_uid']
    qset_id = ann['query_set']

    qset_uuid = f'{annotation_uid}_{qset_id}'
    qset_pred = preds_inter_lit[qset_uuid]
    pred_tree_lit.setdefault(video_uid, {}).setdefault(clip_uid, {}).setdefault(annotation_uid, {})[qset_id] = qset_pred

print(pformat(pred_tree_lit[list(pred_tree_lit.keys())[0]])[:1000])

{'159ac0ad-e5ec-449d-a69e-556655f79b2a': {'bed90ec6-d0fc-4afe-b0b5-534fdfc4324c': {'1': {'ret_bboxes': tensor([[ 1.7362e+00,  4.7274e+01,  7.7746e+01,  1.7153e+02],
        [ 7.6772e-01,  1.6457e+02,  5.5942e+01,  2.8023e+02],
        [ 1.9233e-01,  1.9555e+02,  6.4866e+01,  3.3058e+02],
        ...,
        [-2.1262e+00, -1.2822e+01,  1.7336e+02,  7.9962e+01],
        [ 6.2630e+01,  4.1628e-02,  3.3325e+02,  8.3347e+01],
        [-5.4710e-01, -2.1511e+01,  1.5620e+02,  9.5690e+01]]),
                                                                                         'ret_scores': tensor([-1.8118, -2.5861, -2.3871,  ..., -1.6295, -2.4976, -1.7079])},
                                                                                   '2': {'ret_bboxes': tensor([[4.6435e+02, 4.2506e+02, 6.7272e+02, 6.2217e+02],
        [3.9136e+02, 4.1098e+02, 6.0320e+02, 6.1714e+02],
        [3.4558e+02, 4.7885e+02, 5.8111e+02, 6.7534e+02],
        ...,
        [4.4448e+02, 1.1265e+00, 5.9962e+02, 2

In [4]:
preds = {}
qset_count = 0
for pred_vid in preds_egoloc['videos']:
    video_uid = pred_vid['video_uid']
    for pred_clip in pred_vid['clips']:
        clip_uid = pred_clip['clip_uid']
        for aidx, pred_annot in enumerate(pred_clip['annotations']):
            annotation_uid = list(pred_tree_lit[video_uid][clip_uid].keys())[aidx]
            for qset_id, pred_qset in pred_annot['query_sets'].items():
                bboxes = pred_qset['response_track']
                bboxes = [{
                    'fno': bbox['frame_number'],
                    'x1': bbox['x'], 'y1': bbox['y'],
                    'x2': bbox['x'] + bbox['width'], 'y2': bbox['y'] + bbox['height'],
                } for bbox in bboxes]
                qset_uuid = f"{annotation_uid}_{qset_id}"
                preds[qset_uuid] = bboxes
                qset_count += 1
assert qset_count == len(preds), (qset_count, len(preds))
qset_count, len(anns)

(228, 4504)

In [5]:
pred_tree = {}  # video_uid -> clip_uid -> annotation_uid -> qset_id -> prediction
for ann in anns:
    video_uid = ann['video_uid']
    clip_uid = ann['clip_uid']
    annotation_uid = ann['annotation_uid']
    qset_id = ann['query_set']

    qset_uuid = f'{annotation_uid}_{qset_id}'
    if qset_uuid in preds:
        qset_pred = preds[qset_uuid]
        pred_tree.setdefault(video_uid, {}).setdefault(clip_uid, {}).setdefault(annotation_uid, {})[qset_id] = qset_pred


In [6]:
result = {
    'version': '1.0.5',
    'challenge': 'ego4d_vq2d_challenge',
    'results': {
        'videos': [
        ]
    }
}

for video_uid, clips in pred_tree.items():
    result['results']['videos'].append({
        'video_uid': video_uid,
        'clips': []
    })
    for clip_uid, annotations in clips.items():
        result['results']['videos'][-1]['clips'].append({
            'clip_uid': clip_uid,
            'predictions': []
        })
        for annotation_uid, query_sets in annotations.items():
            result['results']['videos'][-1]['clips'][-1]['predictions'].append({
                'annotation_uid': annotation_uid,
                'query_sets': {}
            })
            for qset_id, qset_pred in query_sets.items():
                fno_s = qset_pred[0]['fno']
                bboxes = np.array([[b['x1'], b['y1'], b['x2'], b['y2']] for b in qset_pred])
                bboxes = [
                    {'fno': fno, 'x1': b[0], 'y1': b[1], 'x2': b[2], 'y2': b[3]}
                    for fno, b in enumerate(bboxes.astype(int).tolist(), start=fno_s)]
                result['results']['videos'][-1]['clips'][-1]['predictions'][-1]['query_sets'][qset_id] = {
                    'bboxes': bboxes,
                    'score': 1.0
                }

In [37]:
result

{'version': '1.0.5',
 'challenge': 'ego4d_vq2d_challenge',
 'results': {'videos': [{'video_uid': 'e14e03f8-13e4-4df2-87b0-e1ad8a175f7c',
    'clips': [{'clip_uid': 'd5935c29-1b8d-417d-9bbb-5ebd47e9256d',
      'predictions': [{'annotation_uid': '3760b439-d94b-4d76-8b39-1bb355800649',
        'query_sets': {'3': {'bboxes': [{'fno': 1292,
            'x1': 1,
            'y1': 1007,
            'x2': 117,
            'y2': 1078},
           {'fno': 1293, 'x1': 0, 'y1': 926, 'x2': 126, 'y2': 1079},
           {'fno': 1294, 'x1': -1, 'y1': 881, 'x2': 80, 'y2': 1079},
           {'fno': 1295, 'x1': 0, 'y1': 811, 'x2': 80, 'y2': 1079},
           {'fno': 1296, 'x1': 2, 'y1': 784, 'x2': 87, 'y2': 1077},
           {'fno': 1297, 'x1': 2, 'y1': 784, 'x2': 87, 'y2': 1077},
           {'fno': 1298, 'x1': 2, 'y1': 784, 'x2': 96, 'y2': 1079},
           {'fno': 1299, 'x1': 2, 'y1': 784, 'x2': 91, 'y2': 1078},
           {'fno': 1300, 'x1': 2, 'y1': 784, 'x2': 93, 'y2': 1078},
           {'fno': 130